In [25]:
import pandas as pd
import yfinance as yf
from datetime import datetime
from pandas.tseries.offsets import BDay

# Load E*TRADE data
df = pd.read_csv("ByStatus_expanded.csv")

# Convert vesting date and filter out future grants
df['Vest Date'] = pd.to_datetime(df['Vest Date'], errors='coerce')
cutoff_date = datetime(2025, 3, 31)
df = df[df['Vest Date'] <= cutoff_date]
df = df[df['Sellable Qty.'].astype(float) > 0]

# Clean USD values
def parse_usd(val):
    try:
        if isinstance(val, str):
            return float(val.replace('$','').replace(',','').strip())
        return float(val) if pd.notnull(val) else 0
    except Exception:
        return 0

# Caches for FX rates and peak prices
fx_cache = {}
peak_cache = {}

# Get USD to INR on vesting date (fallback to previous business day)
def get_usd_to_inr(row):
    date_obj = row['Vest Date']
    try:
        if date_obj.weekday() >= 5:  # Saturday/Sunday
            date_obj -= BDay(1)
        date_str = date_obj.strftime('%d-%b-%Y')
        if date_str in fx_cache:
            return fx_cache[date_str]
        fx = yf.Ticker("USDINR=X")
        hist = fx.history(start=date_str, end=date_str)
        rate = hist['Close'].iloc[0] if not hist.empty else 83.20
        fx_cache[date_str] = rate
        return rate
    except Exception:
        return 83.20

# Get peak USD price from vesting date to 31st March 2025
def get_peak_usd_price(row):
    symbol = row['Symbol']
    start = row['Vest Date']
    end = cutoff_date
    try:
        cache_key = (symbol, start.strftime('%d-%b-%Y'))
        if cache_key in peak_cache:
            return peak_cache[cache_key]
        ticker = yf.Ticker(symbol)
        hist = ticker.history(start=start, end=end)
        peak = hist['Close'].max() if not hist.empty else 0
        peak_cache[cache_key] = peak
        return peak
    except Exception:
        return 0

# Get closing price on 31st March 2025
def get_closing_price_on_cutoff(symbol):
    cache_key = f"{symbol}_closing"
    if cache_key in peak_cache:
        return peak_cache[cache_key]
    try:
        ticker = yf.Ticker(symbol)
        hist = ticker.history(start="2025-03-28", end="2025-04-02")  # 5-day window
        if not hist.empty:
            price = hist.loc["2025-03-31"]['Close'] if "2025-03-31" in hist.index else hist['Close'].iloc[-1]
            peak_cache[cache_key] = price
            return price
        return 0
    except Exception:
        return 0
# Apply dynamic values
df['USD to INR'] = df.apply(get_usd_to_inr, axis=1)
df['Peak USD Price'] = df.apply(get_peak_usd_price, axis=1)
df['Closing USD Price'] = df['Symbol'].apply(get_closing_price_on_cutoff)

# ITR2 Schedule FA A3 field mapping
df['Country/Region name'] = 'United States'
df['Country Name and Code'] = '2'
df['Name of entity'] = "Cadence Design Systems Inc."
df['Address of entity'] = "2655 Seely Avenue San Jose CA"
df['ZIP Code'] = '95134'
df['Nature of entity'] = df['Plan Type'].replace({
    'Rest. Stock': 'Restricted Stock Units',
    'ESPP': 'Employee Stock Purchase Plan'
})
df['Date of acquiring the interest'] = df['Vest Date'].dt.strftime('%d/%b/%Y')

# Initial value: FMV × qty × FX rate
def get_initial_value(row):
    try:
        qty = float(row['Sellable Qty.']) if pd.notnull(row['Sellable Qty.']) else 0
        usd_inr = row['USD to INR']
        fmv = 0
        if pd.notnull(row.get('Grant Date FMV')) and parse_usd(row['Grant Date FMV']) > 0:
            fmv = parse_usd(row['Grant Date FMV'])
        elif pd.notnull(row.get('Purchase Date FMV')) and parse_usd(row['Purchase Date FMV']) > 0:
            fmv = parse_usd(row['Purchase Date FMV'])
        else:
            fmv = parse_usd(row['Est. Market Value']) / qty if qty else 0
        return fmv * qty * usd_inr
    except Exception:
        return 0

df['Initial value of the investment'] = df.apply(get_initial_value, axis=1)

# Peak value: highest price × qty × FX rate
df['Peak value of investment during the Period'] = df['Peak USD Price'] * df['Sellable Qty.'].astype(float) * df['USD to INR']

# Closing balance: price on 31st March × qty × FX rate
df['Closing balance'] = df['Closing USD Price'] * df['Sellable Qty.'].astype(float) * df['USD to INR']

# Dividends/credits and sale proceeds
df['Total gross amount paid/credited with respect to the holding during the period'] = 0
df['Total gross proceeds from sale or redemption of investment during the period'] = 0

# Round all monetary values to 2 decimals
df = df.round(2)

# Final ITR2 export
itr2_df = df[[
    'Country/Region name',
    'Country Name and Code',
    'Name of entity',
    'Address of entity',
    'ZIP Code',
    'Nature of entity',
    'Date of acquiring the interest',
    'Initial value of the investment',
    'Peak value of investment during the Period',
    'Closing balance',
    'Total gross amount paid/credited with respect to the holding during the period',
    'Total gross proceeds from sale or redemption of investment during the period'
]]

itr2_df.to_csv("ITR2_Schedule_FA_A3_Etrade.csv", index=False)

/tmp/ipykernel_35103/2406226125.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Vest Date'] = pd.to_datetime(df['Vest Date'], errors='coerce')
